# SVM using linear kernels

In [12]:
%matplotlib nbagg

import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

class LinearSVM:
    def __init__(self, num_samples, C=1, h=0.5, maxIter=1000):
        self.C = C
        self.n = num_samples
        self.h = h
        self.max_iter = maxIter
        self._obj_vals = []
        self._miss_errs = []
        
    def _huber_loss(self, _X, _y, _w):
        yt = _y * np.dot(_X, _w)
        retval = np.where(yt > (1 + self.h), 0, np.where(yt < (1-self.h), 1 - yt, (1+self.h-yt)**2/(4*self.h)))
        return retval
    
    def _huber_gradient(self, _X, _y, _w):
        yt = _y * np.dot(_X, _w)
        minus_yx = -1*(_y*_X.T).T
        minus_h_to_h = (np.multiply(minus_yx.T,(1+self.h-yt)/(2*self.h))).T
        gradient_vector = np.zeros(_X.shape[1])
        for index, _yt in enumerate(yt):
            loss_at_index = np.zeros(_X.shape[1])
            
            if yt[index] < (1-self.h):
                loss_at_index = minus_yx[index]
            elif yt[index] > (1+self.h):
                loss_at_index = loss_at_index
            else:
                loss_at_index = minus_h_to_h[index]
            gradient_vector = gradient_vector + loss_at_index
                
        return gradient_vector
        
    def compute_obj(self,w):
        yt = np.dot(self.data,w)
        obj = np.power(LA.norm(w),2) + (self.C/self.n)*np.dot(self._huber_loss(self.data, self.target, w), np.ones(self.data.shape[0]))
        return obj;
    
    def compute_grad(self, w):
        return 2*w + (self.C/self.n)*self._huber_gradient(self.data,self.target,w)
    
    def fit(self, X, y):
        self.data = np.dot(X, X.T)
        self.target = y
        
        w = np.zeros(self.data.shape[0])
        for iter in range(0, self.max_iter):
            self._obj_vals.append(obj_fn(w))
            gradient = self.compute_grad(w)
            self._miss_errs.append((np.multiply(np.dot(self.data,w), self.target) < 0).sum())
            w = w - eta*gradient
            
        return w

## Observation
The linear svm calculates the dot product between every $ x_{i} $ & $ x_{j} $ in the function fit.  This gram matrix is then used as the input samples for training the SVM.  The rest of the process of a linear kernel is similar to that of the kernel we used in https://github.com/rajegannathan/computational-machine-learning/blob/master/hw5/Gradient%20and%20objective%20functions.ipynb

Due to time constraints I am rewriting gradient-descent in not so generic way.  Major parts are just copied from the original code in the link provided above.

## Generating Gaussian samples

In [10]:
def generate_gaussian_samples(n_samples, m_dimension):
    C = np.random.uniform(low=-1,high=1,size=(m_dimension,m_dimension))
    X = np.r_[np.dot(np.random.randn(int(n_samples/2), m_dimension), C), 
              np.dot(np.random.randn(int(n_samples/2), m_dimension), C) + np.ones(m_dimension)]
    
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    y = np.ones(n_samples)
    y[:int(n_samples/2)] = -1
    return X, y

## Testing my-svm implementation